In [56]:
import requests
import os
import re
import codecs
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

In [57]:
animeTitle=[]
animeType=[]
animeNumEpisode=[]
releaseDate=[]
endDate=[]
animeNumMembers=[]
anime_info=[]
animeScore=[]
animeUsers=[]
animeRank=[]
animePopularity=[]
animeDescription=[]
animeRelated=[]
animeCharacters=[]
animeVoices=[]

In [58]:
def parse_time(date_list):
    # cheack the date is store as [Month, Day, Year]
    # there is date store as [empty] if film (end date), [?] if still airing
    if len(date_list) == 3:
        date_string = " ".join(date_list).replace(',', '')
        # %b = for abbreviate month
        new_date = datetime.strptime(date_string, '%b %d %Y')
    else:
        # return empty
        new_date = None
    return new_date

In [59]:
for page in tqdm(range(0,1)): 
    folder = "/page"+str(page+1)
    for i in range(1,51):
        #Download
        path = "Folder_with_page" + str(folder) + "/anime_" + str(50*page+i) + ".html"
        file = codecs.open(path, "r", "utf-8")
        soup = BeautifulSoup(file, 'html.parser')
        print(path)
        #animeTitle
        animeTitle.append("".join(soup.find_all('strong')[0].contents[0]))
        for h2 in soup.select('h2:has(+div)'):
            if h2.text == "Information" :
                for inform in h2.find_all_next("div", attrs = {"class": "spaceit_pad"}, limit = 4):
                    #animeType
                    if inform.contents[1].string == "Type:":
                        animeType.append( inform.get_text(separator=" ", strip=True).split()[-1] )
                    #animeNumEpisode
                    if inform.contents[1].string == "Episodes:":
                        animeNumEpisode.append(int(inform.get_text(separator=" ", strip=True).split()[-1]))
                    #releaseDate&endDate    
                    if inform.contents[1].string == "Aired:":
                        releaseDate.append(parse_time(inform.get_text(separator=" ", strip=True).split()[1:4] ))
                        endDate.append( parse_time(inform.get_text(separator=" ", strip=True).split()[5:8]))
        #animeNumMembers
        members = soup.find("span",{"class":"numbers members"})
        #change to int
        animeNumMembers.append("".join(members.find('strong').contents))
        #animeScore
        score = soup.find("div", attrs = {"class": "fl-l score", "data-title": "score"})
        animeScore.append(float(score.contents[0].string))
        #animeRank
        rank= soup.find("span",{"class":"numbers ranked"})
        #change to int
        animeRank.append(("".join(rank.find('strong').contents)))
        #animePopularity  
        popularity = soup.find("span",{"class":"numbers popularity"})
        #change to int
        animePopularity.append("".join(popularity.find('strong').contents))
        
        #animeRelated crash in 15
        #animeRel = []
        #rel1=soup.find_all("table",{"class": "anime_detail_related_anime"})
        #rel2=rel1[0].find_all("tr")
        #rel3=rel2[0].find_all("a")
        #for r in rel3:
            #animeRel.append(r.contents[0])
        #animeRelated.append(animeRel)
        #print(animeRel)
        
        #animeCharacter&Voice  
        animeCharacter = []
        animeVoice = []
        animeS = []
        cvs = soup.find_all("div", {"class": "detail-characters-list clearfix"})
        characters =  cvs[0].find_all("h3", {"class": "h3_characters_voice_actors"})
        voices = cvs[0].find_all("td", {"class": "va-t ar pl4 pr4"})
        
        #staff=cvs[1].find_all("td", {"class": "borderClass"})
        #for s in related: I do in tomorrow morning cuz 03:27 am ahaha
            #print(s.contents[1].contents[0])
            
        for c in characters:
            animeCharacter.append(c.contents[0].contents[0])
        for v in voices:
            animeVoice.append(v.contents[1].contents[0])
        animeCharacters.append(animeCharacter)
        animeVoices.append(animeVoice)
        #animeDesciption
        animeDescription.append(soup.find_all('p',itemprop="description")[0].contents[0])
        

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Folder_with_page/page1/anime_1.html
Folder_with_page/page1/anime_2.html
Folder_with_page/page1/anime_3.html
Folder_with_page/page1/anime_4.html
Folder_with_page/page1/anime_5.html
Folder_with_page/page1/anime_6.html
Folder_with_page/page1/anime_7.html
Folder_with_page/page1/anime_8.html
Folder_with_page/page1/anime_9.html
Folder_with_page/page1/anime_10.html
Folder_with_page/page1/anime_11.html
Folder_with_page/page1/anime_12.html
Folder_with_page/page1/anime_13.html
Folder_with_page/page1/anime_14.html
Folder_with_page/page1/anime_15.html
Folder_with_page/page1/anime_16.html
Folder_with_page/page1/anime_17.html
Folder_with_page/page1/anime_18.html
Folder_with_page/page1/anime_19.html
Folder_with_page/page1/anime_20.html
Folder_with_page/page1/anime_21.html
Folder_with_page/page1/anime_22.html
Folder_with_page/page1/anime_23.html
Folder_with_page/page1/anime_24.html
Folder_with_page/page1/anime_25.html
Folder_with_page/page1/anime_26.html
Folder_with_page/page1/anime_27.html
Folder_wit

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.85s/it]

Folder_with_page/page1/anime_50.html


In [68]:
#animeUsers=[]
#animePopularity=[]
#animeDescription=[]
#animeRelated=[]
#animeCharacters=[]
#animeVoices=[]
d1 = pd.DataFrame(animeTitle, columns = ['animeTitle'])
d2 = pd.DataFrame(animeNumEpisode, columns = ['animeNumEpisode'])
d3 = pd.DataFrame(releaseDate, columns = ['releaseDate'])
d4 = pd.DataFrame(endDate, columns = ['endDate'])
d5 = pd.DataFrame(animeNumMembers, columns = ['animeNumMembers'])
d6 = pd.DataFrame(animeScore, columns = ['animeScore'])
d7 = pd.DataFrame(animeRank, columns = ['animeRank'])
df = pd.concat([d1,d2,d3,d4,d5,d6,d7], axis=1)
print(df)

                                           animeTitle  animeNumEpisode  \
0                    Fullmetal Alchemist: Brotherhood               64   
1                                            Gintama°               51   
2                  Shingeki no Kyojin Season 3 Part 2               10   
3                                         Steins;Gate               24   
4                            Fruits Basket: The Final               13   
5                                            Gintama'               51   
6                              Hunter x Hunter (2011)              148   
7                                Ginga Eiyuu Densetsu              110   
8                                 Gintama': Enchousen               13   
9                                  Gintama: The Final                1   
10                                           Gintama.               12   
11                         3-gatsu no Lion 2nd Season               22   
12                                    

In [ ]:
for i in range(len(df)):
    with open('anime_tsv')
    #and so on

In [51]:
print(animeRelated[4])

['Fruits Basket', 'Fruits Basket 2nd Season', 'Fruits Basket: Prelude']


In [184]:
print(type(animeTitle[0]))
print(type(animeType[0]))
print(type(animeNumEpisode[0]))
print((animeNumMembers[0]))
print(4,type(animeNumMembers[0]))
print(4,type(animeScore[0]))
print(type(releaseDate[0]))
print(type(animeRank[1]))
print(len(animeRank))


<class 'str'>
<class 'str'>
<class 'int'>
2,676,066
4 <class 'str'>
4 <class 'float'>
<class 'datetime.datetime'>
<class 'str'>
50
